In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
class VideoReader:
    def __init__(self):
        pass

    def record(self, callback):
        vid = cv.VideoCapture(0)

        while True:
            ret, frame = vid.read()
            callback(frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break

In [27]:
from abc import ABC
from abc import abstractmethod

class FaceDetector(ABC):
    def __init__(self):
        pass
        
    @abstractmethod
    def detect(self, frame):
        pass

In [69]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

class MediaPipeModels:
    blaze_face_short_range='./blaze_face_short_range.tflite'
    oil_paint ='./face_stylizer_oil_painting.task'
    color_sketch = './face_stylizer_color_sketch.task'

class MediaPipeFaceDetector(FaceDetector):
    def __init__(self, model):
        self.model = model

    def detect(self, frame):
        #make face detector with mediapipe
        BaseOptions = mp.tasks.BaseOptions
        FaceDetector = mp.tasks.vision.FaceDetector
        FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
        VisionRunningMode = mp.tasks.vision.RunningMode

        # Create a face detector instance with the image mode:
        options = FaceDetectorOptions(
            base_options=BaseOptions(model_asset_path=self.model),
            running_mode=VisionRunningMode.IMAGE)
        with FaceDetector.create_from_options(options) as detector:
            # Load the input image from a numpy array.
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            face_detector_result = detector.detect(mp_image)
            return face_detector_result.detections

class MediaPipeFaceStyler:
    def __init__(self, model):
        self.model = model

    def style(self, frame):
        BaseOptions = mp.tasks.BaseOptions
        Facestylizer = mp.tasks.vision.FaceStylizer
        FacestylizerOptions = mp.tasks.vision.FaceStylizerOptions

        # Create a face stylizer instance with the image mode:
        options = FacestylizerOptions(
            base_options=BaseOptions(model_asset_path=self.model))
        with Facestylizer.create_from_options(options) as stylizer:
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            face_stylizer_result = stylizer.stylize(mp_image)
            return face_stylizer_result.numpy_view()

In [95]:


def bounding_box_crop(frame, bounding_box):
    origin_x = int(bounding_box.origin_x)
    origin_y = int(bounding_box.origin_y)
    final_x = int(bounding_box.width)
    final_y = int(bounding_box.height)
    return frame[origin_y:origin_y+final_y, origin_x:origin_x+final_x]

def bounding_box_paint(frame, bounding_box):
    origin_x = int(bounding_box.origin_x)
    origin_y = int(bounding_box.origin_y)
    final_x = int(bounding_box.width)
    final_y = int(bounding_box.height)
    cv.rectangle(frame, (origin_x, origin_y), (origin_x + final_x, origin_y + final_y), (0, 255, 0), 2)


first_time = False

def callback(frame):
    global first_time
    global stylized_face
    detector = MediaPipeFaceDetector(MediaPipeModels.blaze_face_short_range)
    stylizer = MediaPipeFaceStyler(MediaPipeModels.color_sketch)
    #stylized = stylizer.style(frame)
    #print(stylized)
    detections = detector.detect(frame)
    for x in detections:
        bounding_box = x.bounding_box
        keypoints = x.keypoints
        #bounding_box_paint(frame, bounding_box)
        cropped = bounding_box_crop(frame, bounding_box)
        canny = cv.Canny(cropped, 50, 100)
        #canny to rgb
        canny = cv.cvtColor(canny, cv.COLOR_GRAY2RGB)
        #MAKE BORDERS OVERLAP NOT REPLACE
        frame[bounding_box.origin_y:bounding_box.origin_y+bounding_box.height, bounding_box.origin_x:bounding_box.origin_x+bounding_box.width] = canny

    cv.imshow('frame', frame)

VideoReader().record(callback)

KeyboardInterrupt: 